In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import pickle

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

root_dir = os.getcwd() + "\\data\\reorganized"
categories = os.listdir(root_dir)
width = 224
height = 224
dim = (width, height)
data = []


In [29]:
def make_data():
    for category in categories:
        path = os.path.join(root_dir, category)
        label = categories.index(category)

        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)

            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, dim)
                image = np.array(image, dtype=np.float32)
                data.append([image, label])
            except Exception as e:
                pass

    pik = open('pickle/ham10000classifier.pickle', 'wb')
    pickle.dump(data, pik)
    pik.close()

In [30]:
# make_data()

In [25]:
def load_data():
    pick = open('pickle/ham10000classifier.pickle', 'rb')
    data = pickle.load(pick)
    pick.close()

    feature = []
    labels = []

    for img, label in data:
        feature.append(img)
        labels.append(label)

    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    feature = feature / 255.0
    return [feature, labels]

In [26]:
(feature, labels) = load_data()

In [30]:
from ModelResNet import create_model
from tensorflow.keras.optimizers import Adam

lr = 0.0001
model = create_model()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=lr),
              metrics=['acc'])
inputs = feature
targets = labels
fold_no = 1

from KFoldTrain import KFoldtrain

KFoldtrain(num_folds=3,inputs=feature,targets=labels,model=model,categories=categories)
